In [33]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer

# Caminho para o arquivo JSON
file_path = '/content/jogos_detalhe.json'

# ==============================================================================
# Seção 1: Carregamento e Preparação dos Dados
# ==============================================================================

# Carregar os dados do arquivo JSON
with open(file_path, 'r') as file:
    data = json.load(file)
df = pd.json_normalize(data)

# Pré-processamento: Conversão de listas em strings separadas por vírgula
cat_cols_list = ['mecanicas', 'categorias', 'temas']
for col in cat_cols_list:
    df[col] = df[col].apply(lambda x: ', '.join([str(i['nm_'+col[:-1]]) for i in x]) if isinstance(x, list) else x)

# Adicionando imputadores para colunas numéricas e categóricas
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='constant', fill_value='desconhecido')

# Atualizando o preprocessor com imputadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', num_imputer), ('scaler', StandardScaler())]), ['qt_jogadores_min', 'qt_jogadores_max', 'vl_tempo_jogo', 'idade_minima']),
        ('cat', Pipeline(steps=[('imputer', cat_imputer), ('encoder', OneHotEncoder(handle_unknown='ignore'))]), ['mecanicas', 'categorias', 'temas'])
    ])

# ==============================================================================
# Seção 2: Treinamento do Modelo de Clustering
# ==============================================================================

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)  # Defina explicitamente n_init para evitar o aviso
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('cluster', kmeans)])

# Seleção das colunas relevantes para o clustering e treinamento do modelo
cols_for_clustering = ['qt_jogadores_min', 'qt_jogadores_max', 'vl_tempo_jogo', 'idade_minima', 'mecanicas', 'categorias', 'temas']
df_clustering = df[cols_for_clustering]
pipeline.fit(df_clustering)

# Adicionar a coluna de cluster ao DataFrame original
df['cluster'] = pipeline.predict(df_clustering)

# Definindo popularity_cols antes de calcular a pontuação de popularidade
popularity_cols = ['qt_quer', 'qt_favorito', 'qt_jogou', 'qt_tem', 'qt_teve']
df['popularity_score'] = df[popularity_cols].sum(axis=1)

# ==============================================================================
# Seção 3: Função de Recomendação
# ==============================================================================

def recomendar_jogos(base, nome=None, mecanicas=None, categorias=None, temas=None):
    """
    Função para recomendar jogos com base em combinações de nome, mecânicas, categorias e temas.

    Parâmetros:
    - base (DataFrame): DataFrame contendo os dados dos jogos.
    - nome (str, opcional): Nome do jogo para recomendação baseada em cluster.
    - mecanicas (list, opcional): Lista de mecânicas para filtro.
    - categorias (list, opcional): Lista de categorias para filtro.
    - temas (list, opcional): Lista de temas para filtro.

    Retorna:
    - DataFrame com as top recomendações com base na popularidade.
    """
    recomendacoes = base.copy()

    if nome:
        filtro_nome = base['nm_jogo'].str.lower() == nome.lower()
        if filtro_nome.any():
            jogo_selecionado = base[filtro_nome].iloc[0]
            cluster = jogo_selecionado['cluster']
            recomendacoes = base[base['cluster'] == cluster]
        else:
            print(f"Nenhum jogo encontrado com o nome: {nome}")
            return pd.DataFrame()

    # Remover o jogo pesquisado das recomendações
    if nome:
        recomendacoes = recomendacoes[recomendacoes['nm_jogo'].str.lower() != nome.lower()]

    # Filtrar por mecânicas, categorias e temas, se fornecidos
    if mecanicas:
        recomendacoes = recomendacoes[recomendacoes['mecanicas'].str.contains('|'.join(mecanicas), case=False, na=False)]
    if categorias:
        recomendacoes = recomendacoes[recomendacoes['categorias'].str.contains('|'.join(categorias), case=False, na=False)]
    if temas:
        recomendacoes = recomendacoes[recomendacoes['temas'].str.contains('|'.join(temas), case=False, na=False)]

    if recomendacoes.empty:
        print("Nenhuma recomendação encontrada.")
        return recomendacoes

    # Ordenar as recomendações pela pontuação de popularidade e retornar os top 3
    top_recomendacoes = recomendacoes.sort_values(by='popularity_score', ascending=False).head(3)
    return top_recomendacoes[['nm_jogo', 'popularity_score']]

# ==============================================================================
# Seção 4: Exemplos de Uso da Função de Recomendação
# ==============================================================================

# Exemplo de chamada da função
# resultado_nome = recomendar_jogos(df, tipo='nome', valor='Stone Age')
# print(resultado_nome)

# Exemplo de chamada da função para mecânica
# resultado_mecanica = recomendar_jogos(df, tipo='mecanica', valor='Alocação de Trabalhadores')
# print(resultado_mecanica)

# Nota: Remova os comentários das chamadas de função para executá-las e ver os resultados.

In [52]:
# Exemplo de pesquisa por nome
resultado_nome = recomendar_jogos(df, nome='Catan: O Jogo')
print(resultado_nome)

            nm_jogo  popularity_score
598           Dixit           15235.0
766  The Resistance           14470.0
327        Munchkin           12097.0


In [57]:
# Exemplo de pesquisa combinada
resultado_combinado = recomendar_jogos(df, mecanicas=['RPG'], temas=['Policial'])
print(resultado_combinado)

                                               nm_jogo  popularity_score
11181  Mansions of Madness (2ª Edição): Além do Limiar            2106.0
12835  Mansions of Madness (2ª Edição): Ruas de Arkham            2032.0
59                                 Mansions of Madness            1866.0
